In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
## Import libraries

import os
import random
import cv2
import json
import numpy as np
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
# Global variables:

n_timesteps = 125    # n_timesteps should be the same as maxlen for padding
vector_size = 75    # same as feature_size
max_frame = n_timesteps
cells = 2
n_features = vector_size
size = 250

## Common functions utelized in both reading json files and images

In [0]:
# input = other_0_000000000000_rendered.png
# output = other_0
def get_file_name(file_name):
    parts = file_name.split('_')
    return parts[0] + '_' + parts[1]

# input = 'push' or 'other'
# output = 1 or 0
def encode_label(label):
    if 'push' in label:
        return 1
    elif 'other' in label:
        return 0
    else:
        raise ValueError('label not supported')

# input = array
# output = normalized_array
def normalized_value(array):
    mean = array.mean()
    std = array.std()
    norm_arr = array - mean
    norm_arr /= std
    return norm_arr


# input = video directory
# output = list of video name: [other_0, other_1, ..., push_0, push_1, ..]
def get_vidlist(video_dir):
    vid_list = []
    for file in os.listdir(video_dir):
        vid_name = file.split('.')[0]
        vid_list.append(vid_name)

    return vid_list
    

# input = vid_list
# output = train_list and val_list shuffled by name
def get_train_val_list(vid_list, seed=6, split=0.8):
    other = []
    push = []
    for vid_name in vid_list:
        if 'other' in vid_name:
            other.append(0)
        elif 'push' in vid_name:
            push.append(0)

    split_other = int(len(other) * split)
    split_push = int(len(push) * split)

    train_vid_list = vid_list[0:split_other] + vid_list[len(other):len(other) + split_push]
    val_vid_list = vid_list[split_other:len(other)] + vid_list[len(other) + split_push:]

    random.Random(seed).shuffle(train_vid_list)
    random.Random(seed).shuffle(val_vid_list)

    return train_vid_list, val_vid_list


# reshaping the target to align the shape of the x_train
# input = list
# output = expanded array on axis=0
def add_first_dim(list):
    arr = np.asarray(list)
    return np.expand_dims(arr, axis=0)


# reshaping the target to align the shape of the x_train
# input = list
# output = expanded array on axis=0
def add_first_dim(list):
    arr = np.asarray(list)
    return np.expand_dims(arr, axis=0)

## Reading Json files with Generator
Although Json files are generally light enough and may not need generator. However, since the model is an API model and it is fit as model.fit_generator(), both input needs to be read in the same way.

Notes about generator:
1. The generator, generates data by batchsize (n-timesteps) and hence save memory
2.  The generator needs to return a tuple of (x_train, target)
3.  For some reason generator in Keras needs While loop (Stackoverflow)
4. A generator by itself is an object and only could be called through a for loop

In [0]:
## Note: in openpose, videos with no figure returns jsons with empty people object. These frames are waived when reading jsons.
# So, The file names should be pass to reading images so the corresponding frames also get waived while reading images


# input = json_file directory
# output = a dic of file_name where json people are empty.
def get_invalid_frames(json_dir):
    no_fig_files = {}
    for file in os.listdir(json_dir):
        file_path = json_dir + '/' + file
        temp = json.load(open(file_path))
        if len(temp['people']) == 0:
            parts = file.split('_')
            img_name = parts[0] + '_' + parts[1] + '_' + parts[2] + '_rendered.png' #change to png
            file_name = get_file_name(file)
            no_fig_files.setdefault(file_name, []).append(img_name)

    return no_fig_files

# input = json_file directory , video_name (e.g: other_0)
# output = list of list of features (vector of 75 size) relating to all frames corresponding to video_name 
def read_data_from_landmarks(json_dir,vid_name):
    landmarks = []
    for file in os.listdir(json_dir):
        if vid_name == get_file_name(file):
            file_path = json_dir + '/' + file
            temp = json.load(open(file_path))
            if len(temp['people']) == 0:
                pass
            else:
                value = temp['people'][0]['pose_keypoints_2d']
                norm_value = [float(i) / sum(value) for i in value]     # normalize the landmark features
                landmarks.append(norm_value)

    return landmarks


#### Land_mark generator which yeild a tuple of padded_landmarks and corresponding label
# This generator will later be called in model.fite_generator()
def landmark_generator(vid_list, json_dir, pad_len=n_timesteps):
    while True:
        for vid_name in vid_list:
            landmark = read_data_from_landmarks(json_dir, vid_name)
            padded = pad_sequences([landmark], dtype='float32', maxlen=pad_len, padding='post')
            batch = (padded, add_first_dim(encode_label(vid_name)))
            yield batch

## Reading captured frames with Generator
The default function of keras for image generator (ImageDataGenerator) is not used since I could not handle times series in this function. Hence a generator is scripted.


In [0]:
# input = image_file
# output = img_array
def read_img(img_file):
    img = cv2.imread(img_file)
    img = cv2.resize(img, dsize=(size, size), interpolation=cv2.INTER_CUBIC)
    return img


# input= files in frame_dir and vid_name
# output = arrays of all frames of one video, exluding invalid frames that are waived already in reading json files (frames with no figure)
def get_array_of_frames(frame_dir, vid_name, json_dir):
    images = []
    invalid_frames = get_invalid_frames(json_dir)     # invalid frames of json needs to be waived in reading frames
    for file in os.listdir(frame_dir):
        if vid_name == get_file_name(file):

            if vid_name in invalid_frames:
                if file in invalid_frames[vid_name]:
                    continue    #pass

            array = read_img(frame_dir + "/" + file)
            norm_array = normalized_value(array)
            images.append(norm_array)

    return images


#### image generator which yeild a tuple of padded_arrays of images and corresponding label
# This generator will later be called in model.fite_generator()
def image_generator(vid_list, frame_dir, json_dir, pad_len=n_timesteps):
    while True:
        for vid_name in vid_list:
            frames = get_array_of_frames(frame_dir, vid_name, json_dir)
            padded = pad_sequences([frames], dtype='float32', maxlen=pad_len, padding='post')
            batch = (padded, add_first_dim(encode_label(vid_name)))
            yield batch

## Model Configuration
Using Functional API to creat 2 branches for 2 types of input (json and image)

In [0]:
## import libraries

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
#from keras.layers.convolutional import Conv3D
#from keras.layers.pooling import MaxPooling3D
from keras.layers import LSTM
from keras.layers.merge import concatenate
from keras.applications import MobileNet
from keras.layers import TimeDistributed


In [0]:
# Json
landmark_input = Input(shape=(max_frame, vector_size),
               name='landmark')           
lstm1 = LSTM(units=cells, dropout=0.1, recurrent_dropout=0.5, return_sequences=True)(landmark_input)
flat1 = Flatten()(lstm1)

# image
image_input = Input(shape=(max_frame, size, size, 3), name = 'image')
base_model = MobileNet(input_shape=(size, size, 3),
                       include_top=False,
                       weights='imagenet',
                       input_tensor=None,
                       pooling='avg',
                       classes=2)

im = TimeDistributed(base_model)(image_input)
im = LSTM(units=cells, return_sequences=True)(im)
flat2 = Flatten()(im)

merge = concatenate([flat1, flat2])
#dr = Dropout(0.5)(merge)

hidden = Dense(1, activation='relu')(merge)
output = Dense(1, activation='sigmoid')(hidden)    # softmax

model = Model([landmark_input,image_input], output)
print(model.summary())

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              (None, 50, 250, 250, 0                                            
__________________________________________________________________________________________________
landmark (InputLayer)           (None, 50, 75)       0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 50, 1024)     3228864     image[0][0]                      
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 50, 2)        624         landmark[0][0]                   
____________________________________________________________________________________________

## Multiple generator
Unlike model.fit() that can accept multiple inputs, model.fit_generator() can not accept multiple generators by default. Hence a function is added to make it work with multiple generators. Source_code: https://github.com/keras-team/keras/issues/8130

In [0]:
def generate_multiple_generator(genX1, genX2):
    while True:
      try:
        X1i = next(genX1)
        X2i = next(genX2)
        yield [X1i[0], X2i[0]], X1i[1]
      except StopIteration:
        print('This was the problemmmmmmmmmmmmmmmmmmm')
        break

## Calling Generator objects and Get input data

In [0]:
base_dir = "/content/drive/My Drive/Google_Colab"
json_dir = base_dir + '/' + 'openpose_json'
frame_dir = base_dir + '/' + 'video_frames'
video_dir = base_dir + '/' + 'video'

vid_list = get_vidlist (video_dir)
train_list, val_list = get_train_val_list(vid_list, split=0.8)

# calling json generator for train and validation
train_json_generator = landmark_generator(train_list, json_dir)
val_json_generator = landmark_generator(val_list, json_dir)

# calling image generator for train and validation
train_img_generator = image_generator(train_list, frame_dir,json_dir)
val_img_generator = image_generator(val_list, frame_dir,json_dir)

# calling generate_multiple_generator to get train_gen and val_gen
train_gen = generate_multiple_generator(train_json_generator,train_img_generator)
val_gen = generate_multiple_generator(val_json_generator, val_img_generator)

In [0]:
# fit the model with two inputs!

epochs = 1

# if __name__ == '__main__':
history=model.fit_generator(generator = train_gen,
                        steps_per_epoch=len(train_list),
                        epochs = epochs,
                        validation_data = val_gen,
                        validation_steps = len(val_list),
                        use_multiprocessing=True,
                        workers =1000)   #, workers = 2     and use_multiprocessing=True for Colab   



Epoch 1/1


/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


In [0]:
from keras.utils import plot_model

plot_model(model, to_file='model_8_API.png')